In [1]:
#Inisialisasi Library

from PIL import Image
from PIL import ImageDraw

import glob
import pandas as pd
from skimage.feature import greycomatrix, greycoprops
from skimage.transform import resize
import matplotlib.pyplot as plt
import numpy as np
import os
import cv2
import math

ubah8bit_3bit = [i // 32 for i in range(256)]


Bad key "text.kerning_factor" on line 4 in
C:\Users\ASUS\Anaconda3\envs\tensorflowproject\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution


In [2]:
# Fungsi-fungsi
# Proses membuka gambar

def read_file(mypath):
    file_name = []
    label = []
    full_path = []

    for path, subdirs, files in os.walk(mypath):
        for name in files:
            full_path.append(os.path.join(path, name))
            label.append(path.split('/')[-1])
            
    return full_path, label

def load_img(img_path):
    img = Image.open(img_path).convert('L')
    hasil_img = img.resize([224,224], Image.NEAREST)
    return hasil_img

def print_img(img):
    plt.imshow(img, cmap='gray')
    
def getLabels(path):
    label = os.listdir(path)
    return label
    
# Preprocessing Gambar
    
def getAllColorIMG(img):
    AllColorIMG = list(img.getdata())
    return AllColorIMG

def ubahKe3bit(AllColorIMG):
    i = 0
    
    for ColorIMG in AllColorIMG:
        AllColorIMG[i] = ubah8bit_3bit[ColorIMG]
        i += 1
        
    return AllColorIMG

def ubahKeMatriks(AllColorIMG):
    matriksIMG = [] 
    i = 0
    
    dimensiGambar = int(math.sqrt(len(AllColorIMG)))
    
    for row in range(dimensiGambar):
        newRowMat = []
        for col in range(dimensiGambar):
            newRowMat.append(AllColorIMG[i])
            i += 1
        matriksIMG.append(newRowMat)
        
    return matriksIMG

def featureExtract(img):
    colorIMG = getAllColorIMG(img)
    _3BitColor = ubahKe3bit(colorIMG)
    matriksIMG = ubahKeMatriks(_3BitColor)
    
    resultOfGLCM = []
    glcm = greycomatrix(matriksIMG, distances=[1], angles=[0], levels=256, symmetric=False, normed=False)
    
    komponen = ['dissimilarity', 'contrast', 'homogeneity', 'energy', 'correlation', 'ASM']
    
    for i in komponen:
        resultOfGLCM.append(greycoprops(glcm, i)[0][0])
    
    return resultOfGLCM

In [3]:
# Memproses File Training dengan mengekstrak fiturnya sebelum dimodelkan

train_path = 'dataset_magang/Training/'
imgType = '*.jpg'

PathOf_trainIMG = []
LabelOf_trainIMG = []
Dissimilarity = []
Contrast = []
Homogeneity = []
Energy = []
Correlation = []
ASM = []

labels = getLabels(train_path)

for label in labels:
    full_paths = glob.glob(train_path + label + '/' + imgType)
    for full_path in full_paths:
        PathOf_trainIMG.append(full_path)
        LabelOf_trainIMG.append(label)
        trainIMG = load_img(full_path)
        featureOfIMG = featureExtract(trainIMG)
        Dissimilarity.append(featureOfIMG[0])
        Contrast.append(featureOfIMG[1])
        Homogeneity.append(featureOfIMG[2])
        Energy.append(featureOfIMG[3])
        Correlation.append(featureOfIMG[4])
        ASM.append(featureOfIMG[5])

columns = ["Path", "Dissimilarity", "Contrast", "Homogeneity", "Energy", "Correlation", "ASM" , "Label"]
variables = {"Path":PathOf_trainIMG, "Dissimilarity":Dissimilarity, "Contrast":Contrast, "Homogeneity":Homogeneity, "Energy":Energy, "Correlation":Correlation, "ASM":ASM ,"Label":LabelOf_trainIMG}
df_train = pd.DataFrame(variables, columns = columns)
df_train.to_csv('DataTrainingBatik.csv', index=False)     

# <center>Metode Decision Tree<center>

In [4]:
#Proses Modelling Data Training

from sklearn.model_selection import train_test_split

X = df_train.iloc[:, 1:6].values  
y = df_train.iloc[:, 7].values
X_trainDT, X_testDT, y_trainDT, y_testDT = train_test_split(X, y, test_size=0.3, random_state=0)

In [5]:
# Melakukan klasifikasi dengan decision tree kemudian menetapkan model decision tree dengan data training

from sklearn.tree import DecisionTreeClassifier

classifierDT = DecisionTreeClassifier(criterion="entropy", max_depth=4)
classifierDT.fit(X_trainDT, y_trainDT)
y_predDT = classifierDT.predict(X_testDT)

In [6]:
# Menampilkan confusion matrix dan classification_report untuk melihat performasi algoritma melalui recall, presisi,F1-Score,dll

from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

print("Algoritma Decision Tree")
print("Confusion Matrix:\n", confusion_matrix(y_testDT, y_predDT), "\n")
print("Classification Measure\n", classification_report(y_testDT, y_predDT))
print("Accuracy:",accuracy_score(y_testDT, y_predDT))

Algoritma Decision Tree
Confusion Matrix:
 [[50 32]
 [ 6 73]] 

Classification Measure
               precision    recall  f1-score   support

         Cap       0.89      0.61      0.72        82
       Tulis       0.70      0.92      0.79        79

    accuracy                           0.76       161
   macro avg       0.79      0.77      0.76       161
weighted avg       0.80      0.76      0.76       161

Accuracy: 0.7639751552795031


# <center>Metode k-NN<center>

In [7]:
X_trainKNN, X_testKNN, y_trainKNN, y_testKNN = train_test_split(X, y, test_size=0.3, random_state=0)

from sklearn.preprocessing import StandardScaler  
scaler = StandardScaler()  
scaler.fit(X_trainKNN)

X_trainKNNs = scaler.transform(X_trainKNN)  
X_testKNNs = scaler.transform(X_testKNN)

#Melakukan klasifikasi dengan k neighbors=5
from sklearn.neighbors import KNeighborsClassifier  
classifier = KNeighborsClassifier(n_neighbors=5)  
classifier.fit(X_trainKNNs, y_trainKNN)

#Melakukan prediksi dari klasifikasi
y_predKNN = classifier.predict(X_testKNNs)

#Menampilkan confussion matriks guna melihat recall, presisi, F!Score, dll
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print("Algoritma k-NN")
print ("Confusion Matrix:\n", confusion_matrix(y_testKNN, y_predKNN), "\n")
print ("Classification Measure\n", classification_report(y_testKNN, y_predKNN))
print("Accuracy:",accuracy_score(y_testKNN, y_predKNN))

Algoritma k-NN
Confusion Matrix:
 [[73  9]
 [ 5 74]] 

Classification Measure
               precision    recall  f1-score   support

         Cap       0.94      0.89      0.91        82
       Tulis       0.89      0.94      0.91        79

    accuracy                           0.91       161
   macro avg       0.91      0.91      0.91       161
weighted avg       0.91      0.91      0.91       161

Accuracy: 0.9130434782608695


# <center>Metode Naive Bayes

In [8]:
from sklearn.naive_bayes import GaussianNB

X_trainNB, X_testNB, y_trainNB, y_testNB = train_test_split(X, y, test_size=0.3, random_state=0)

#membuat Gaussian Classifier kemudian menetapkan model Gaussian Classifier menggunakan set training
gnb = GaussianNB()
gnb.fit(X_trainNB, y_trainNB)
y_predNB = gnb.predict(X_testNB) #memprediksi respon dari dataset test

print ("Algoritma Naive Bayes")
print ("Confusion Matrix:\n", confusion_matrix(y_testNB, y_predNB), "\n")
print ("Classification Measure\n", classification_report(y_testNB, y_predNB))
print("Accuracy:",accuracy_score(y_testNB, y_predNB))

Algoritma Naive Bayes
Confusion Matrix:
 [[56 26]
 [16 63]] 

Classification Measure
               precision    recall  f1-score   support

         Cap       0.78      0.68      0.73        82
       Tulis       0.71      0.80      0.75        79

    accuracy                           0.74       161
   macro avg       0.74      0.74      0.74       161
weighted avg       0.74      0.74      0.74       161

Accuracy: 0.7391304347826086


# <center>Metode Support Vector Machines (SVMs)

In [9]:
from sklearn import svm
from sklearn.svm import SVC  


kernels = ['Polynomial', 'RBF', 'Sigmoid','Linear']
#A function which returns the corresponding SVC model
def getClassifier(ktype):
    if ktype == 0:
        # Polynomial kernal
        return SVC(kernel='poly', degree=8, gamma="auto")
    elif ktype == 1:
        # Radial Basis Function kernal
        return SVC(kernel='rbf', gamma="auto")
    elif ktype == 2:
        # Sigmoid kernal
        return SVC(kernel='sigmoid', gamma="auto")
    elif ktype == 3:
        # Linear kernal
        return SVC(kernel='linear', gamma="auto")

for i in range(4):
    # Separate data into test and training sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=0)
# Train a SVC model using different kernal
    svclassifier = getClassifier(i) 
    svclassifier.fit(X_train, y_train)
# Make prediction
    y_pred = svclassifier.predict(X_test)
# Evaluate our model
    print("Evaluation:", kernels[i], "kernel")
    print(classification_report(y_test,y_pred))
    print ("Accuracy:",accuracy_score(y_test, y_pred))


Evaluation: Polynomial kernel
              precision    recall  f1-score   support

         Cap       0.97      0.43      0.59        82
       Tulis       0.62      0.99      0.76        79

    accuracy                           0.70       161
   macro avg       0.80      0.71      0.68       161
weighted avg       0.80      0.70      0.68       161

Accuracy: 0.7018633540372671
Evaluation: RBF kernel
              precision    recall  f1-score   support

         Cap       0.83      0.63      0.72        82
       Tulis       0.69      0.86      0.77        79

    accuracy                           0.75       161
   macro avg       0.76      0.75      0.74       161
weighted avg       0.76      0.75      0.74       161

Accuracy: 0.7453416149068323
Evaluation: Sigmoid kernel
              precision    recall  f1-score   support

         Cap       0.34      0.32      0.33        82
       Tulis       0.33      0.35      0.34        79

    accuracy                           0.34 

In [10]:
# Mengimport library yang akan digunakan untuk melakukan variasi parameter dengan Grid Search
from sklearn.model_selection import GridSearchCV

#Create a dictionary called param_grid and fill out some parameters for kernels, C and gamma
param_grid = {'C': [0.1,1, 10, 100], 'gamma': [1,0.1,0.01,0.001]}

# Membangun model dengan grid search
grid = GridSearchCV(SVC(),param_grid,refit=True,verbose=2)
grid.fit(X_train,y_train)

print(grid.best_estimator_)

Fitting 5 folds for each of 16 candidates, totalling 80 fits
[CV] C=0.1, gamma=1 ..................................................
[CV] ................................... C=0.1, gamma=1, total=   0.0s
[CV] C=0.1, gamma=1 ..................................................
[CV] ................................... C=0.1, gamma=1, total=   0.0s
[CV] C=0.1, gamma=1 ..................................................
[CV] ................................... C=0.1, gamma=1, total=   0.0s
[CV] C=0.1, gamma=1 ..................................................
[CV] ................................... C=0.1, gamma=1, total=   0.0s
[CV] C=0.1, gamma=1 ..................................................
[CV] ................................... C=0.1, gamma=1, total=   0.0s
[CV] C=0.1, gamma=0.1 ................................................
[CV] ................................. C=0.1, gamma=0.1, total=   0.0s
[CV] C=0.1, gamma=0.1 ................................................
[CV] ...........

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s


[CV] ................................. C=0.1, gamma=0.1, total=   0.0s
[CV] C=0.1, gamma=0.01 ...............................................
[CV] ................................ C=0.1, gamma=0.01, total=   0.0s
[CV] C=0.1, gamma=0.01 ...............................................
[CV] ................................ C=0.1, gamma=0.01, total=   0.0s
[CV] C=0.1, gamma=0.01 ...............................................
[CV] ................................ C=0.1, gamma=0.01, total=   0.0s
[CV] C=0.1, gamma=0.01 ...............................................
[CV] ................................ C=0.1, gamma=0.01, total=   0.0s
[CV] C=0.1, gamma=0.01 ...............................................
[CV] ................................ C=0.1, gamma=0.01, total=   0.0s
[CV] C=0.1, gamma=0.001 ..............................................
[CV] ............................... C=0.1, gamma=0.001, total=   0.0s
[CV] C=0.1, gamma=0.001 ..............................................
[CV] .

[CV] ................................ C=100, gamma=0.01, total=   0.0s
[CV] C=100, gamma=0.01 ...............................................
[CV] ................................ C=100, gamma=0.01, total=   0.0s
[CV] C=100, gamma=0.01 ...............................................
[CV] ................................ C=100, gamma=0.01, total=   0.0s
[CV] C=100, gamma=0.01 ...............................................
[CV] ................................ C=100, gamma=0.01, total=   0.0s
[CV] C=100, gamma=0.001 ..............................................
[CV] ............................... C=100, gamma=0.001, total=   0.0s
[CV] C=100, gamma=0.001 ..............................................
[CV] ............................... C=100, gamma=0.001, total=   0.0s
[CV] C=100, gamma=0.001 ..............................................
[CV] ............................... C=100, gamma=0.001, total=   0.0s
[CV] C=100, gamma=0.001 ..............................................
[CV] .

[Parallel(n_jobs=1)]: Done  80 out of  80 | elapsed:    1.0s finished


In [11]:
print(grid.best_estimator_)

SVC(C=100, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=1, kernel='rbf', max_iter=-1,
    probability=False, random_state=None, shrinking=True, tol=0.001,
    verbose=False)


In [12]:
# Menerima respon hasil prediksi
grid_pred = grid.predict(X_test)
# Menampilkan confusion matrix dan classification_report untuk melihat performasi algoritma melalui recall, presisi,F1-Score,dll
print ("Confusion Matrix:\n", confusion_matrix(y_test, grid_pred), "\n")
print ("Classification Measure\n", classification_report(y_test, grid_pred))
print ("Accuracy:",accuracy_score(y_test, grid_pred))

Confusion Matrix:
 [[71 11]
 [ 7 72]] 

Classification Measure
               precision    recall  f1-score   support

         Cap       0.91      0.87      0.89        82
       Tulis       0.87      0.91      0.89        79

    accuracy                           0.89       161
   macro avg       0.89      0.89      0.89       161
weighted avg       0.89      0.89      0.89       161

Accuracy: 0.8881987577639752


# <center>Metode Random Forest

In [13]:
from sklearn.ensemble import RandomForestClassifier

X_trainRF, X_testRF, y_trainRF, y_testRF = train_test_split(X, y, test_size=0.3, random_state=0)

# Membangun model dengan Random Forest Classifier dengan data training
model = RandomForestClassifier(n_estimators=100, random_state=0)
model.fit(X_trainRF, y_trainRF)
y_predRF = model.predict(X_testRF)

# Menampilkan confusion matrix dan classification_report untuk melihat performasi algoritma melalui recall, presisi,F1-Score,dll
print ("Algoritma Random Forest")
print ("Confusion Matrix:\n", confusion_matrix(y_testRF, y_predRF), "\n")
print ("Classification Measure\n", classification_report(y_testRF, y_predRF))
print ("Accuracy:",accuracy_score(y_testRF, y_predRF))

Algoritma Random Forest
Confusion Matrix:
 [[75  7]
 [ 8 71]] 

Classification Measure
               precision    recall  f1-score   support

         Cap       0.90      0.91      0.91        82
       Tulis       0.91      0.90      0.90        79

    accuracy                           0.91       161
   macro avg       0.91      0.91      0.91       161
weighted avg       0.91      0.91      0.91       161

Accuracy: 0.906832298136646


In [14]:
# Validasi dengan menggunakan file validasi

val_path = 'dataset_magang/Validasi/'
imgType = '*.jpg'

PathOf_valIMG = []
LabelOf_valIMG = []
Dissimilarity = []
Contrast = []
Homogeneity = []
Energy = []
Correlation = []
ASM = []
feature = []

labels = getLabels(val_path)

for label in labels:
    full_val_paths = glob.glob(val_path + label + '/' + imgType)
    for full_val_path in full_val_paths:
        PathOf_valIMG.append(full_val_path)
        LabelOf_valIMG.append(label)
        valIMG = load_img(full_val_path)
        featureOfIMG = featureExtract(valIMG)
        Dissimilarity.append(featureOfIMG[0])
        Contrast.append(featureOfIMG[1])
        Homogeneity.append(featureOfIMG[2])
        Energy.append(featureOfIMG[3])
        Correlation.append(featureOfIMG[4])
        ASM.append(featureOfIMG[5])
        feature.append(featureOfIMG)

    
columns = ["Path", "Dissimilarity", "Contrast", "Homogeneity", "Energy", "Correlation", "ASM" , "Label"]
variables = {"Path":PathOf_valIMG, "Dissimilarity":Dissimilarity, "Contrast":Contrast, "Homogeneity":Homogeneity, "Energy":Energy, "Correlation":Correlation, "ASM":ASM ,"Label":LabelOf_valIMG}
df_val = pd.DataFrame(variables, columns = columns)
df_val.to_csv('DatavalidasiBatik.csv', index=False)     

In [15]:
# Percobaan 2 validasi dengan KNN

X = df_train.iloc[:, 1:6].values  
y = df_train.iloc[:, 7].values
val_data = df_val.iloc[:, 1:6].values  
label_val_data = df_val.iloc[:, 7].values

from sklearn.preprocessing import StandardScaler  
scaler = StandardScaler()  
scaler.fit(X)

X_trainKNN = scaler.transform(X)  
X_testKNN = scaler.transform(val_data)
y_testKNN = label_val_data

#Melakukan klasifikasi dengan k neighbors=5
from sklearn.neighbors import KNeighborsClassifier  
classifier = KNeighborsClassifier(n_neighbors=5)  
classifier.fit(X_trainKNN, y)

#Melakukan prediksi dari klasifikasi
y_predKNN = classifier.predict(X_testKNN)

#Menampilkan confussion matriks guna melihat recall, presisi, F!Score, dll
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print("Algoritma k-NN")
print ("Confusion Matrix:\n", confusion_matrix(y_testKNN, y_predKNN), "\n")
print ("Classification Measure\n", classification_report(y_testKNN, y_predKNN))
print("Accuracy:",accuracy_score(y_testKNN, y_predKNN))

Algoritma k-NN
Confusion Matrix:
 [[110   8]
 [ 17 121]] 

Classification Measure
               precision    recall  f1-score   support

         Cap       0.87      0.93      0.90       118
       Tulis       0.94      0.88      0.91       138

    accuracy                           0.90       256
   macro avg       0.90      0.90      0.90       256
weighted avg       0.90      0.90      0.90       256

Accuracy: 0.90234375
